In [2]:
!cd

c:\Users\GarvikaRajawat\Desktop\Yashu's Project\Loan-Advisory-System


In [1]:
!pip install -q streamlit pandas python-dotenv langchain langchain-google-genai chromadb pytesseract pillow pypdf


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-exporter-otlp-proto-http 1.36.0 requires opentelemetry-exporter-otlp-proto-common==1.36.0, but you have opentelemetry-exporter-otlp-proto-common 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.36.0 requires opentelemetry-proto==1.36.0, but you have opentelemetry-proto 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.36.0 requires opentelemetry-sdk~=1.36.0, but you have opentelemetry-sdk 1.37.0 which is incompatible.
opentelemetry-instrumentation 0.57b0 requires opentelemetry-semantic-conventions==0.57b0, but you have opentelemetry-semantic-conventions 0.58b0 which is incompatible.


In [7]:
!pip install -U sentence-transformers -q

In [3]:
!pip install -U langchain-community -q


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-exporter-otlp-proto-http 1.36.0 requires opentelemetry-exporter-otlp-proto-common==1.36.0, but you have opentelemetry-exporter-otlp-proto-common 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.36.0 requires opentelemetry-proto==1.36.0, but you have opentelemetry-proto 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.36.0 requires opentelemetry-sdk~=1.36.0, but you have opentelemetry-sdk 1.37.0 which is incompatible.


In [32]:
import streamlit as st
import pandas as pd
import os
from dotenv import load_dotenv
import sys
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders import PyPDFLoader, TextLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import pytesseract
from PIL import Image
from langchain_community.embeddings import JinaEmbeddings



In [33]:
# Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in environment variables. Please check your .env file.")


In [10]:
with open("Knowledge-base/Policy_doc.md", "r", encoding="utf-8") as file:
    markdown_document = file.read() 

headers_to_split_on = [
    ("#", "Header 1"),
    # ("##", "Header 2"),
    # ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers = False)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(metadata={'Header 1': 'Loan Advisory Policy'}, page_content='# Loan Advisory Policy'),
 Document(metadata={'Header 1': 'Purpose'}, page_content='# Purpose\nThis policy outlines the principles, standards, and procedures that guide the provision of loan advisory services.\nIt ensures clients receive transparent, ethical, and compliant advice tailored to their financial needs.'),
 Document(metadata={'Header 1': 'Scope'}, page_content='# Scope\nThis policy applies to all employees, contractors, and representatives involved in providing loan advisory services.\nIt covers all client engagements, loan products, and advisory-related communications.'),
 Document(metadata={'Header 1': 'Definitions'}, page_content='# Definitions\n- **Loan Advisory**: Professional guidance provided to clients regarding loan products, terms, and obligations.\n- **Client**: Any individual or entity seeking loan advisory services.\n- **Adviser**: Authorized staff member responsible for providing guidance.')

In [ ]:
!pip install -U langchain_ollama

In [37]:
print(os.getenv("JINA_API_KEY"))

jina_ad21598f6a354596ad805fde515f4efakP4__wM07ACl1zx33NujW6eg8dYh


In [39]:
# from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(model="llama3")
# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001", google_api_key=GEMINI_API_KEY
# )
embeddings = JinaEmbeddings(jina_api_key=os.getenv("JINA_API_KEY"), model_name="jina-embeddings-v3")

In [17]:
md_header_splits[0].page_content

'# Loan Advisory Policy'

In [18]:
sentences = []
for i in range(len(md_header_splits)):
    sentences.append(md_header_splits[i].page_content)

In [40]:
vectorstore = Chroma.from_documents(
    md_header_splits, embeddings, persist_directory="./chroma_db"
)
vectorstore.persist()

C:\Users\GarvikaRajawat\AppData\Local\Temp\ipykernel_20228\1748713543.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [47]:
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-lite-latest", google_api_key=GEMINI_API_KEY, temperature=0.7
)

In [49]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5})
)
query = "What are the eligibility criteria in the Loan Advisory Policy?"
response = qa_chain.invoke({"query": query})
print(response["result"])

The Loan Advisory Policy mentions an "Assessment of financial profile and loan eligibility" as part of the Advisory Process, but it does not explicitly detail the specific eligibility criteria themselves.


In [50]:
# streamlit_app.py
import streamlit as st
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# -----------------------------
# Streamlit UI Setup
# -----------------------------
st.set_page_config(page_title="RAG Chatbot", page_icon="💬", layout="centered")
st.title("💬 RAG Chatbot with ChromaDB")

# Initialize session state for chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

2025-09-27 23:26:51.747 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:51.749 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:51.861 
  command:

    streamlit run c:\Users\GarvikaRajawat\Desktop\MCP\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-27 23:26:51.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:51.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:51.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:51.871 Session state does not function when running a script without `stre

In [51]:
# -----------------------------
# Chat Input
# -----------------------------
with st.form("chat_form", clear_on_submit=True):
    user_input = st.text_input("Ask me anything:", "")
    submit = st.form_submit_button("Send")

if submit and user_input:
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Get response from RAG QA
    with st.spinner("Thinking..."):
        result = qa_chain.run(user_input) #changes qa_chain 
    
    # Add bot response to chat history
    st.session_state.messages.append({"role": "assistant", "content": result})

# -----------------------------
# Display Chat History
# -----------------------------
for message in st.session_state.messages:
    if message["role"] == "user":
        st.markdown(f"**You:** {message['content']}")
    else:
        st.markdown(f"**Bot:** {message['content']}")

2025-09-27 23:26:54.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:54.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:54.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:54.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:54.103 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:54.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:54.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-27 23:26:54.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar